# Bot Python Treat Excel / Send Email

## All imports

In [1]:
import pycountry
import os
import pandas as pd
import win32com.client as win32
from currency_converter import CurrencyConverter
cc = CurrencyConverter()
from datetime import datetime

## Read the excel file, process the data and create new files

In [2]:
# Open the Excel file as a DataFrame.
df = pd.read_excel('./Base/ds_salaries.xlsx')

# Remove the work_year only to represent some data that is not relevant for that moment.
df = df.drop('work_year', axis=1)  

# Alter abbreviation to full name in 'experience_level'.
for i, level in enumerate(df['experience_level']):
    try:
        if level == 'EN':
            df.at[i, 'experience_level'] = 'Junior'
        elif level == 'MI':
             df.at[i, 'experience_level'] = 'Intermediate'
        elif level == 'SE':
             df.at[i, 'experience_level'] = 'Expert'
        else:
             df.at[i, 'experience_level'] = 'Director'
    except: 
        continue

# Alter abbreviation to full name in 'company_size' using another way.
df.loc[(df['company_size'] == 'L'), 'company_size'] = 'Large'
df.loc[(df['company_size'] == 'M'), 'company_size'] = 'Medium'
df.loc[(df['company_size'] == 'S'), 'company_size'] = 'Small'
        
# Using Pycountry lib to get country according to region.        
for i, emp_residence, com_loc in zip(df.iloc[:, 0], df.iloc[:, 7], df.iloc[:, 9]):
    df.at[i, 'employee_residence'] = pycountry.countries.get(alpha_2 = emp_residence).name
    df.at[i, 'company_location'] = pycountry.countries.get(alpha_2 = com_loc).name 
   
# Remove Unnamed Column. 
df = df.drop(df.columns[0], axis = 1)
        
# Rename column to more professional template.
old_names = df.columns.tolist()
new_names = []

for name in old_names:
    if name == 'salary_in_usd':
        new_names.append('Salary in USD')
    else:
        new_names.append(name.replace('_', ' ').title())

df.columns = new_names

# Create the column "Salary in BRL" using the currency_converter lib and adjust decimal places to 0.
brl_value = cc.convert(1, 'USD', 'BRL')
df['Salary in BRL'] = df['Salary in USD'] * brl_value
df['Salary in BRL'] = df['Salary in BRL'].apply(lambda x: '{:.0f}'.format(x))

# Change the order of columns
df = df[['Experience Level', 'Job Title', 'Employment Type', 'Remote Ratio', 'Salary Currency', 'Salary', 
         'Salary in USD', 'Salary in BRL', 'Employee Residence', 'Company Location', 'Company Size']]

# Create new Excel sheets splitted by 'Experience Level'
for experience_level in df['Experience Level'].unique():
    df.loc[df['Experience Level'] == experience_level].to_excel(
            f'./Raw Splitted Sheets/{experience_level}.xlsx', index=False, sheet_name= experience_level)

# Return the DataFrame with all changes.
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './Raw Splitted Sheets/Intermediate.xlsx'

## Formatting the new Excel files

In [ ]:
# Get all Excel files created in folder.
files = os.listdir('./Raw Splitted Sheets')

# While loop to format all files that were found in the folder 
while(files):
    
    # Get first element in files array.
    file = files.pop(0)

    # Get the name of file without extension.
    name_file = file.split('.')[0]

    # Open the Excel file as a DataFrame.
    df_file = pd.read_excel(f'./Raw Splitted Sheets/{file}')

    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(f'./Modified Splitted Sheets/{name_file}_Modified.xlsx', engine='xlsxwriter')

    # Convert the dataframe to an XlsxWriter Excel object.
    df_file.to_excel(writer, index=False, sheet_name= name_file)

    # Get the xlsxwriter objects from the dataframe writer object.
    workbook  = writer.book
    worksheet = writer.sheets[name_file]

    # Create custom format to Excel style
    warning_format = workbook.add_format({'bold': True, 'font_color': 'red', 'num_format' : '#,##0.00'})
    number_format = workbook.add_format({'num_format' : '0'})
    currency_format = workbook.add_format({'num_format' : '#,##0.00'})
    header1_format = workbook.add_format({'bold': True, 'text_wrap': False, 'valign': 'top',
                                     'fg_color': '#5DADE2', 'border': 1})
    header2_format = workbook.add_format({'bold': True, 'text_wrap': False, 'valign': 'top',
                                     'fg_color': '#C1F54E', 'border': 1})
    header3_format = workbook.add_format({'bold': True, 'text_wrap': False, 'valign': 'top',
                                     'fg_color': '#FF4F34', 'border': 1})

    # Set the column 'Salary in BRL' in number.
    worksheet.set_column('H:H', 30, number_format)

    # Set salary columns in currency.
    worksheet.set_column('F:H', 30, currency_format)

    # Set diferent format of Headers.
    for col, col_value in enumerate(df_file.columns.values):
        if (col < 4 ):
            worksheet.write(0, col, col_value, header1_format)
        elif (col < 8):
            worksheet.write(0, col, col_value, header2_format)
        else:
            worksheet.write(0, col, col_value, header3_format)

    # Set conditional formatting for 'Salary' column where salary is below average.
        # Uses the 'Salary in USD' column because the values are then in the same currency, 
        # but change in the 'Salary' column.
    salary_USD_mean = '{:.0f}'.format(df_file['Salary in USD'].mean())
    salary_values = df_file['Salary'].values
    column_index = df_file.columns.get_loc('Salary')

    for row, row_value in enumerate(df_file['Salary in USD']):
        if (int(row_value) < int(salary_USD_mean)):
            worksheet.write(row + 1, column_index, salary_values[row], warning_format)

    writer.save()

## Email sender

In [ ]:
# Get Outlook Application with win32 lib.
outlook = win32.Dispatch('outlook.application')

# Get the contact file.
df_contacts = pd.read_excel('./Contacts/Contacts.xlsx')

# Get all Excel files created in folder.
files = os.listdir('./Modified Splitted Sheets')

# Get all Excel files name.
file_names = []
for file in files:
    file_names.append(file.split('_')[0])

# Checks if match Email list and files to send.
contact_areas = df_contacts['Responsible Area'].unique().tolist()
contact_areas.remove('All')

if (sorted(contact_areas) != sorted(file_names)):
    resp = input('The contact list and areas to send email do not match. Continue? [Yes / No]').lower()
    if (resp.startswith('n')):
        raise TypeError('The contact list and areas to send email do not match. Continue?')
    
# Loop to create emails as drafts.
while(files):
    file = files.pop(0)
    
    # Set the email configure
    filename = file.split('_')[0]
    filtered_contacts = df_contacts[(df_contacts == filename).any(axis=1)]
    copy_contacts = df_contacts[(df_contacts == 'All').any(axis=1)]
    email_to = '; '.join(filtered_contacts['Email Andress'].tolist())
    email_copy = '; '.join(copy_contacts['Email Andress'].tolist())
    hour = datetime.today().strftime('%H')
    greeting = 'Good Morning' if (int(hour) < 12) else 'Good Afternoon'
    recipient_names = ', '.join(filtered_contacts['First Name'].tolist())
    
    # Create the email
    email = outlook.CreateItem(0)
    email.To = email_to
    email.CC = email_copy
    email.Attachments.Add(f'{os.getcwd()}/Modified Splitted Sheets/{file}')
    email.Subject = f'Data science job salary reports level: {filename}'
    email.HTMLBody = f'''
    <div style="font-family: Calibri">
        <p>{greeting} {recipient_names} </p>
        
        <p>Attached is the spreadsheet with the salary reports of the Data Science level <b>{filename}</b></p>
        
        <p>Salaries that are below average are highlighted in <b style="color: 'red'">red and bold</b></p>
        
        <p>Regards, </br><b><a href="https://github.com/steniodr" target="_blank">Stenio D. Rapchan</a></b></p>
        <p>Using Bot Excel/Mail in Python</p>
    </div>    
    '''

    email.Save()
    
print('All emails created as draft!')